### Takes in xlsx file with tabs, calculates correlation moving averages for 20 years, does some groupby visualizations.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xlrd #needed for getting xlsx sheet names
from sklearn import preprocessing
from IPython.display import Image
####################################################################
%run helperFunctions.ipynb #loads defs from other file,cleaner code 
####################################################################
%matplotlib inline
plt.style.use('seaborn-whitegrid')

ERROR:root:File `'helperFunctions.ipynb.py'` not found.


In [2]:
def fromXLSXtoDF(sheetNames_param, xlsxData_param):
    allDF = []
    for i in sheetNames_param:
        # print the sheet name (not necessary) but print statements give valuable feedback to the programmer and are used everywhere 
        # in the development process.
        print("Reading in sheet:", i)
        # so read each sheet named "i", and grab only columns 0 and 5 (which contain the relevant data)
        df = pd.read_excel(xlsxData_param, sheet_name=i, usecols=[0, 5])
        # set the timestamp to the index, this makes time series analysis easier.
        df = df.set_index("Date")
        # calculate diff and percent change, should be done per year
        # diff is the first difference, so the change in data from one day to the next.  
        df["PremiumDiff"] = df["Premium"].diff(1)
        # percent change is the change from one day to the next.
        df["PremiumPrctChange"] = df["Premium"].pct_change(1)
        # add the dataframe to the list of dataframes
        allDF.append(df)
    # concatenate across column (axis=0), use the index as key)
    result = pd.concat(allDF, axis=0, join='outer')
    # change names of columns to the sheetnames
    # result.columns = sheetNames
    return(result)

### Small function to get names of sheets,  If all the names in the sheets are used, then you could do something like: sheetNames = getSheetNames.  If not all names are used, you can remove those names that are not used by hand or programatically: keepThese = [x for x in sheetNames if x not in dontKeep], so dontKeep looks something like: dontKeep = [sheetIDontWant1, sheetIDontWant2, ...]

In [3]:
def getSheetNames(xlsxData_param):
    xls = xlrd.open_workbook(xlsxData_param, on_demand=True)
    return(xls.sheet_names())

The code below takes the required names of the Excel sheets and the Excel file and runs the function fromXLSXtoDF defined in the above function.

In [ ]:
print(getSheetNames('../data/Pna Prem vs Brl - Jeff.xlsx'))

In [6]:
!pwd

/home/jeff001/Travis_priceCorrelations/code/notebooks


In [7]:
print(getSheetNames('../../data/raw/Pna_Prem_Upload_Sheet_no_empty_rows.xlsx'))

['Pna Feb17', 'Pna Mar17', 'Pna Apr17', 'Pna May17', 'Pna Jun17', 'Pna Jul17', 'Pna Aug17', 'Pna Feb18', 'Pna Mar18', 'Pna Apr18', 'Pna May18', 'Pna Jun18', 'Pna Jul18', 'Pna Aug18', 'Pna Sep18', 'Pna Oct18', 'Pna Nov18', 'Pna Dec18', 'Pna Feb19', 'Pna Mar19', 'Pna Apr19', 'Pna May19', 'Pna Jun19', 'Pna Jul19', 'Pna Aug19', 'Pna Sep19', 'Pna Oct19', 'Pna Nov19', 'Pna Dec19', 'Pna Feb20', 'Pna Mar20', 'Pna Apr20', 'Pna May20', 'Pna Jun20', 'Pna Jul20', 'Pna Aug20', 'Pna Sep20', 'Pna Oct20', 'Pna Feb21', 'Pna Mar21', 'Pna Apr21', 'Pna May21', 'Pna Jun21', 'Pna Jul21']


In [ ]:
sheetNames = ['SB Par Mar17', 'SB Par Mar18', 'SB Par Mar19']

In [ ]:
#Calculate correlations for raw data

#Read in the name of the Excel file
xlsxData = '../data/Pna Prem vs Brl - Jeff.xlsx'

#Run the function
sbYears = fromXLSXtoDF(sheetNames, xlsxData)

#Drop missing values
sbYears = sbYears.dropna()

The code below loads the Brazilian currency data.  We didn't write a function because we only do these steps once.

In [ ]:
#Read in BRL dta

#Read in data from the same Excel file in sheet named 'Brl'
brl = pd.read_excel('../data/Pna Prem vs Brl - Jeff.xlsx', 'Brl')

#Set Date (GMT) as the index, which makes times series analysis easier
brl = brl.set_index('Date (GMT)')

#Re-naming index to Date in order to merge 
brl.index.name = 'Date' 

#Calculate the percentage change, just like in the function 
brl['BrlPrctChange'] = brl["Last"].pct_change(1)

#Calculate the first differeence, just like in the function
brl['BrlDiff'] = brl["Last"].diff(1)

#Take only the relevant columns of the Pandas dataframe
brl = brl[["Last", "BrlDiff", "BrlPrctChange"]]

#Rename the columns called Last to Brl
brl = brl.rename(columns = {'Last':'Brl'})

#Drop missing values
brl = brl.dropna()

#Show me the results
brl.head(30)

Combine the two data frames created above

In [ ]:
# Merge data
merD = pd.merge(sbYears,brl,how='inner',on='Date')
merD.describe()

The data is not continuous over the years, so only take those days for which data is available

In [ ]:
df1 = merD
year2016 = df1['2016-06-07': '2017-03-15']
print("2016:", year2016.shape)  #print states to find out how much data each dataframe holds
year2017 = df1['2017-06-05': '2018-03-15']
print("2017:", year2017.shape)
year2018 = df1['2018-06-05': '2019-03-15']
print("2018:", year2018.shape)
year2019 = df1['2019-06-05': '2020-03-15']
print("2019:", year2019.shape)  #2019 is empty

---------------------
---------------------

Take a look at the data.  The difference in scale of the data distorts any visual representation in the realationship.

In [ ]:
year2017[['PremiumPrctChange', 'BrlPrctChange']].plot(figsize=(15,7))

In [ ]:
def corrWindow(prem_param, brl_param, mvWindows):
    corrMv = []
    for i in range(0,prem1.size - mvWindows):
        cor1 = prem_param[i:i+mvWindows].corr(brl_param[i:i+mvWindows])
        corrMv.append(cor1)

    #reindex using subset of the dates
    df = pd.DataFrame({'moving_Corr': corrMv},
                  index = prem1.index[0:prem1.size - mvWindows])    
    return(df)

In [ ]:
# Run the code above on the premium and Brazil data.

prem1 = df1["PremiumPrctChange"]['2018-06-05': '2019-03-15']
brl1  = df1["BrlPrctChange"]['2018-06-05': '2019-03-15']
moveWindows = 20    
df = corrWindow(prem1, brl1, moveWindows)    
df.plot(figsize=(15,7))        

In [ ]:
prem1 = df1["Premium"]['2018-06-05': '2019-03-15']
brl1  = df1["Brl"]['2018-06-05': '2019-03-15']
moveWindows = 20    
df = corrWindow(prem1, brl1, moveWindows)    
df.plot(figsize=(15,7))  

In [ ]:
# the resample method is super handy, the "m" stands for monthly (there is a "w").  So, resample the data per month and take the monthly median (or mean or whatever).

merD[["PremiumPrctChange", "BrlPrctChange"]].resample("m").median().plot(figsize=(15,7))

In [ ]:
# scaleDF is imported from helperFunctions
sDf = scaleDF(merD)
sDf[["PremiumPrctChange", "BrlPrctChange"]].resample("m").median().plot(figsize=(15,7))

In [ ]:
# Subsetting the months for which we have data.
syear2016 = sDf['2016-06-05': '2017-03-15']
syear2017 = sDf['2017-06-05': '2018-03-15']
syear2018 = sDf['2018-06-05': '2019-03-15']
syear2019 = sDf['2019-06-05': '2020-03-15']

# Let's plot SPX and VIX cumulative returns with recession overlay
plot_cols = ['Premium', 'Brl']
fig, axes = plt.subplots(2,1, figsize=(15,7), sharex=True, sharey=False)
syear2016[plot_cols].plot(subplots=True, ax=axes)
fig, axes = plt.subplots(2,1, figsize=(15,7), sharex=True, sharey=False)
syear2017[plot_cols].plot(subplots=True, ax=axes)
fig, axes = plt.subplots(2,1, figsize=(15,7), sharex=True, sharey=False)
syear2018[plot_cols].plot(subplots=True, ax=axes)

In [ ]:
# Let's plot SPX and VIX cumulative returns with recession overlay
plot_cols = ['Premium']
fig, axes = plt.subplots(1,1, figsize=(15,3), sharex=True, sharey=True)
syear2016[plot_cols].plot(subplots=True, ax=axes)
fig, axes = plt.subplots(1,1, figsize=(15,3), sharex=True, sharey=True)
syear2017[plot_cols].plot(subplots=True, ax=axes)
fig, axes = plt.subplots(1,1, figsize=(15,3), sharex=True, sharey=True)
syear2018[plot_cols].plot(subplots=True, ax=axes)

In [ ]:
# to try, merge by month, day doesn't work